In [1]:
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random


# Set the seed for NumPy
np.random.seed(42)

# Set the seed for the built-in random module
random.seed(42)

In [2]:
 def get_number_from_pal(str_num):
    return np.fromiter((int(bit) for bit in str_num), dtype=np.int8)

In [3]:
# Generating palindrome data

def is_pal(X):
    return X == X[::-1]

def generate_10_bit_binary_numbers():
    binary_numbers = []

    for i in range(2**10):
        binary_str = format(i, '010b')
        binary_numbers.append(binary_str)

    return binary_numbers


# Generate and store all 10-bit binary numbers
binary_numbers_list = generate_10_bit_binary_numbers()


Y_VALUES = []
X_VALUES = []

# Print and/or use the generated binary numbers as needed
for binary_number in binary_numbers_list:
    #print(binary_number)
    if is_pal(binary_number) == True:
        X_VALUES.append(get_number_from_pal(binary_number))
        Y_VALUES.append(1)
    else:
        X_VALUES.append(get_number_from_pal(binary_number))
        Y_VALUES.append(0)

In [4]:
X = np.array(X_VALUES)
Y = np.array(Y_VALUES)
print("X = ",X.shape)
print("Y = ",Y.shape)


X =  (1024, 10)
Y =  (1024,)


In [5]:
def precision(result, reference):
    result = np.array(result)
    reference = np.array(reference)
    result = np.atleast_1d(result.astype(np.bool_))
    reference = np.atleast_1d(reference.astype(np.bool_))

    tp = np.count_nonzero(result & reference)
    fp = np.count_nonzero(result & ~reference)

    try:
        precision = tp / float(tp + fp)
    except ZeroDivisionError:
        precision = 0.0

    return precision

In [6]:
# prompt: generate a blank class definition for a MLP in numpy

class MLP:
  def __init__(self, input, hidden):
    self.w1 = np.random.rand(input, hidden)
    # self.w1 = np.random.rand(10, hidden)
    self.w2 = np.random.rand(hidden, 1)

    self.hidden = hidden
    self.input = input

  def relu(self, x):
    return np.maximum(x,0)

  def relu_prime(self,x):
    if x>0:
      return 1
    else:
      return 0

  def sigmoid(self,x):
    # return 1 / (1 + np.exp(-x))
    return np.exp(-np.logaddexp(0, -x))


  def sigmoid_prime(self,x):
    z = self.sigmoid(x)
    return z*(1-z)

  def forward(self, x):
    # forward
    z1 = self.w1.T @ x
    a1 = self.relu(z1)

    z2 = self.w2.T @ a1
    a2 = self.sigmoid(z2)

    return a2

  def forward_backward(self, x, y, lr):

    # forward
    z1 = self.w1.T @ x
    a1 = self.relu(z1)

    z2 = self.w2.T @ a1
    a2 = self.sigmoid(z2)

    # backprop
    self.dw1 = np.zeros_like(self.w1)
    self.dw2 = np.zeros_like(self.w2)

    L = -y*np.log(a2 + 1e-6) - (1-y)*np.log(1-a2 + 1e-6)
    # da2 = -y/a2 + (1-y)/(1-a2)

    for i in range(self.hidden):
      self.dw2[i,0] = (-y[0,0]*(1-a2[0,0]) + (1- y[0,0])*a2[0,0]) * a1[i,0]
      #self.dw2[i,0] = da2[0,0] * self.sigmoid_prime(z2[0,0]) * a1[i,0]

    for i in range(self.input):
      for j in range(self.hidden):
        # self.dw1[i,0] = da2[0,0] * self.sigmoid_prime(z2[0,0]) * self.w2[j,0] * self.relu_prime(z1[j,0]) * x[i,0]
        self.dw1[i,j] = (-y[0,0]*(1-a2[0,0]) + (1- y[0,0])*a2[0,0])  * self.w2[j,0] * self.relu_prime(z1[j,0]) * x[i,0]

    # update
    self.w1 = self.w1 - lr*self.dw1
    self.w2 = self.w2 - lr*self.dw2

    # print (np.linalg.norm(self.dw1))

    return a2 , L


In [7]:
net = MLP(10, 16)

In [8]:
# eval
THRESHOLD = 0.5
def inference(X_test, Y_test):
    print ('=== Inference ===')
    correct = 0
    ALL_PREDS = []
    ALL_YS = []
    for itr in range(len(X_test)):

        x = X_test[itr:itr+1,:].T
        y = Y_test[itr,0]

        y_pred = net.forward(x)

        y_pred = y_pred[0,0]

        # print (y_pred)

        if y_pred < THRESHOLD:
            y_pred = 0
        else:
            y_pred = 1
        ALL_PREDS.append(y_pred)
        ALL_YS.append(y)
        if (y_pred == y):
            correct +=1

    print(correct)
    return (ALL_PREDS, ALL_YS)

In [9]:
! mkdir history

mkdir: cannot create directory ‘history’: File exists


In [10]:

# Set the number of folds (k)
k = 4  # You can adjust this based on your preference

# Get the number of samples
num_samples = X.shape[0]

# Shuffle indices
indices = np.arange(num_samples)
np.random.shuffle(indices)

# Split indices into k folds
fold_indices = np.array_split(indices, k)

# Iterate over the folds
for i in range(k):
    print("FOLD = ",i)
    test_indices = fold_indices[i]
    train_indices = np.concatenate([fold_indices[j] for j in range(k) if j != i])

    X_train, X_test = X[train_indices], X[test_indices]
    Y_train, Y_test = Y[train_indices], Y[test_indices]

    Y_test = Y_test.reshape(Y_test.shape[0],1)
    Y_train = Y_train.reshape(Y_train.shape[0],1)
    # Your training and testing code goes here
    # For example, you can train your model using X_train, Y_train, and then evaluate on X_test, Y_test
    # print("X_test = ",X_test.shape," Y_test",Y_test.shape)

    TOTAL_EPOCHS = 1000
    LOSS_LIST = []
    for epoch in range(TOTAL_EPOCHS):
        total = 0
        for itr in range(len(X_train)):
            x = X_train[itr:itr+1,:].T
            # y = Y[itr:itr+1,:]
            y = Y_train[itr:itr+1,:]
            # print("x shape = ",x.shape," y shape = ",y.shape)

            y_pred, L = net.forward_backward(x,y,1e-2)

            # print(y_pred, '- gt:', y)
            # print(y)
            # print (L)
            # print("=====")

            total += L
        LOSS_LIST.append(total[0][0])
        print ('Epoch: ',epoch, '/ Fold: ',i,' Loss: ', total[0][0])
        # print("--i = ",i)
        with open("history/train_logs_fold_{}.txt".format(str(i)), "a") as text_file:
            text_file.write("{} {:.6f} \n".format(epoch, total[0][0]))
        print ("====")

    ALL_PREDS, ALL_YS = inference(X_test, Y_test)
    print("LOSS LIST = ",LOSS_LIST)
    plt.title('Loss vs epochs')
    # Set axis labels
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.plot(LOSS_LIST)
    plt.savefig('Loss_fold_{}.png'.format(i), bbox_inches='tight')
    plt.close()

    print("ALL PREDS = ",ALL_PREDS)
    print("ALL YS = ",ALL_YS)
    precision_val = precision(ALL_PREDS, ALL_YS)
    print("precision ==> ",precision_val)

    with open("history/precision_fold_{}.txt".format(str(i)), "a") as text_file:
            text_file.write("{:.6f} \n".format(precision_val))
    # if epoch%20==0:
    #     inference(X_test, Y_test)


FOLD =  0
Epoch:  0 / Fold:  0  Loss:  263.1372589299576
====
Epoch:  1 / Fold:  0  Loss:  140.3726575570037
====
Epoch:  2 / Fold:  0  Loss:  139.55352297995492
====
Epoch:  3 / Fold:  0  Loss:  139.0259799451402
====
Epoch:  4 / Fold:  0  Loss:  138.66821471835203
====
Epoch:  5 / Fold:  0  Loss:  138.41361592583883
====
Epoch:  6 / Fold:  0  Loss:  138.22406777910257
====
Epoch:  7 / Fold:  0  Loss:  138.07693097150323
====
Epoch:  8 / Fold:  0  Loss:  137.95831790657937
====
Epoch:  9 / Fold:  0  Loss:  137.859456926151
====
Epoch:  10 / Fold:  0  Loss:  137.7746536380524
====
Epoch:  11 / Fold:  0  Loss:  137.7001130097911
====
Epoch:  12 / Fold:  0  Loss:  137.63324112344992
====
Epoch:  13 / Fold:  0  Loss:  137.57224556292846
====
Epoch:  14 / Fold:  0  Loss:  137.51597448675025
====
Epoch:  15 / Fold:  0  Loss:  137.46328501951643
====
Epoch:  16 / Fold:  0  Loss:  137.41307065788197
====
Epoch:  17 / Fold:  0  Loss:  137.3650839362274
====
Epoch:  18 / Fold:  0  Loss:  137.31

KeyboardInterrupt: 

In [ ]:
y_pred_list = []

for i in range(len(X)):
    x = X[i:i+1,:].T
    y = Y[i,0]

    y_pred = net.forward(x)

    y_pred = y_pred[0,0]

    y_pred_list.append(y_pred)

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y, y_pred_list)

0.5

In [ ]:
# 100  loss:  [[79.19130256]]  - using randn as weights
# 100  loss:  [[26.3380848]] - using rand as weights

In [ ]:
np.sum(Y==1)

111

In [ ]:
np.sum(Y==0)

96

In [ ]:
len(X)
# 96 correct beginning

207

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Create the MLP model
mlp_model = MLPClassifier(hidden_layer_sizes=(16,), max_iter=1000, random_state=42, solver='sgd', learning_rate_init = 1e-2)

# Train the model
mlp_model.fit(X, Y)

# Make predictions on the test set
y_pred = mlp_model.predict(X)

# Calculate accuracy
accuracy = accuracy_score(Y, y_pred)
print(f"Accuracy: {accuracy}")

# You can also access the trained model parameters
print("Trained model parameters:")
print("Input layer size:", mlp_model.n_features_in_)
print("Output layer size:", mlp_model.n_outputs_)
# print("Hidden layer sizes:", mlp_model.hidden_layer_sizes_)


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1098: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 1.0
Trained model parameters:
Input layer size: 10
Output layer size: 1


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
992/1024

0.96875